In [52]:
import torch
import torch.nn as nn
import torch.functional as F

%load_ext autoreload
%autoreload 2

In [62]:
from BLM import BLModel

In [1]:
with open('input.txt', 'r', encoding='utf-8') as fp:
    text = fp.read()

    
print(len(text))

1115394


In [20]:
max([len(line) for line in text.splitlines()])

63

In [3]:
chars = sorted(list(set(text)))
vocab_len = len(chars)
vocab_len

65

In [11]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(encode("hello"))
print(decode([46, 43, 50, 50, 53]))

[46, 43, 50, 50, 53]
hello


In [15]:
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.long)
data.shape

torch.Size([1115394])

In [16]:
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [21]:
block_size = 8

In [65]:
torch.manual_seed(1337)
batch_size = 4

def get_batch(split):
    data = train if split=='train' else val
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [66]:
x_b, y_b = get_batch('train')

In [67]:
model = BLModel(vocab_len)
logits, loss = model(x_b, y_b)
print(logits, loss)

tensor([[ 1.6347, -0.0518,  0.4996,  ...,  0.2432,  1.1519,  0.9950],
        [ 0.3418, -0.9276,  1.2381,  ...,  1.5018, -0.5266,  0.2354],
        [ 0.1479, -0.4333,  0.5203,  ...,  0.3302,  1.5454,  1.3778],
        ...,
        [-0.5693, -0.0735,  0.7743,  ..., -0.0815, -1.1445, -0.0623],
        [ 0.4658, -0.2573, -1.0673,  ...,  1.2439,  1.3471,  1.6910],
        [-0.4553,  0.0139,  0.9309,  ...,  0.0290, -0.7568,  0.8701]],
       grad_fn=<ViewBackward0>) tensor(5.0364, grad_fn=<NllLossBackward0>)
